# Miscellaneous Examples

This notebook demonstrates various features of InteropRouter, including error handling.


In [1]:
import os

from anthropic import AsyncAnthropic
from google import genai
from openai import AsyncOpenAI
from openai.types.responses import EasyInputMessageParam

from interop_router.router import Router
from interop_router.types import ChatMessage, ContextLimitExceededError

router = Router()
openai_client = AsyncOpenAI()
gemini_client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
anthropic_client = AsyncAnthropic()
router.register("openai", openai_client)
router.register("gemini", gemini_client)
router.register("anthropic", anthropic_client)

## Context Limit Errors

When input exceeds a model's context window, InteropRouter raises `ContextLimitExceededError` regardless of provider. The exception includes `provider` (which provider failed) and `cause` (the original exception).


In [2]:
from interop_router.types import SupportedModel

# Generate content large enough to exceed context limits
base_text = "This is a test message with content to fill the context window. "
large_content = base_text * (2_000_000 * 4 // len(base_text) + 1)
messages = [ChatMessage(message=EasyInputMessageParam(role="user", content=large_content))]

models: list[SupportedModel] = ["gpt-5.2", "gemini-3-flash-preview", "claude-haiku-4-5-20251001"]
for model in models:
    try:
        await router.create(input=messages, model=model)
    except ContextLimitExceededError as e:
        print(f"{e.provider}: {type(e.cause).__name__} - {str(e)[:100]}...")

openai: BadRequestError - Error code: 400 - {'error': {'message': 'Your input exceeds the context window of this model. Please...


gemini: ClientError - 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please ...


anthropic: BadRequestError - Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'prompt is...


## Azure OpenAI

Azure OpenAI uses the same API as OpenAI, so you can use the standard `AsyncOpenAI` client with a different `base_url`. The [new v1 API](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/api-version-lifecycle?view=foundry-classic&tabs=python) no longer requires an `api_version` parameter.


In [3]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT", "").rstrip("/")
azure_token = os.getenv("AZURE_OPENAI_AD_TOKEN")

azure_client = AsyncOpenAI(
    base_url=f"{azure_endpoint}/openai/v1/",
    api_key=azure_token,
)

azure_router = Router()
azure_router.register("openai", azure_client)

message = ChatMessage(message=EasyInputMessageParam(role="user", content="Hello!"))
response = await azure_router.create(input=[message], model="gpt-5-mini")

for msg in response.output:
    if msg.message.get("type") == "message":
        content = msg.message.get("content", [])
        if isinstance(content, list):
            for c in content:
                if text := c.get("text"):
                    print(text)

Hello! How can I help you today?
